# Grupowanie algorytmem Birch

*Algorytm grupowania BIRCH (Balanced Iterative Reducing and Clustering using Hierarchies) jest techniką hierarchicznego grupowania danych. Jego główne cechy to efektywność, skalowalność i zdolność do obsługi dużych zbiorów danych. Algorytm ten skupia się na redukcji danych w procesie iteracyjnym, tworząc drzewo CF (Clustering Feature) jako strukturę hierarchiczną.*

## Wczytywanie danych w sparku

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!git clone https://github.com/PiotrMaciejKowalski/BigData2024Project.git
#%cd BigData2024Project
#!git checkout your-branch
#%cd ..

fatal: destination path 'BigData2024Project' already exists and is not an empty directory.


In [3]:
!chmod 755 /content/BigData2024Project/src/setup.sh
!/content/BigData2024Project/src/setup.sh

In [4]:
import sys
sys.path.append('/content/BigData2024Project/src')

In [5]:
from start_spark import initialize_spark
initialize_spark()

In [6]:
from pyspark.sql import SparkSession

In [7]:
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

## Import bibliotek

In [8]:
!pip install datashader
!pip install holoviews hvplot colorcet
!pip install geoviews

In [9]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from collections.abc import Iterable
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
from typing import List, Union
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import datashader as ds
import datashader.transfer_functions as tf
import colorcet as cc
import holoviews as hv
from holoviews.operation.datashader import datashade
import geoviews as gv
import geoviews.tile_sources as gts
from holoviews import opts
from IPython.display import IFrame
from IPython.core.display import display
from bokeh.plotting import show, output_notebook
from sklearn.cluster import Birch
from big_mess.loaders import default_loader, load_single_month, load_anotated, save_to_csv, preprocessed_loader
import pandas as pd
from sklearn.metrics import classification_report

## Tworzenie funkcji

In [10]:
def StandardScaling_on_predefined_attributes(df: pd.DataFrame) -> pd.DataFrame:
  """
  Funkcja służąca do przeskalowania kolumn 'Rainf', 'Evap','AvgSurfT','Albedo','SoilT_40_100cm','GVEG','PotEvap','RootMoist','SoilM_100_200cm' z ramki danych df za pomocą funkcji StandardScaler.
  Parametry:
  - df (DataFrame): Pandas DataFrame zawierająca co najmniej kolumny 'lon', 'lat', 'Rainf', 'Evap','AvgSurfT','Albedo','SoilT_40_100cm','GVEG','PotEvap','RootMoist','SoilM_100_200cm'.
  """
  for col in ['lon', 'lat', 'Rainf', 'Evap','AvgSurfT','Albedo','SoilT_40_100cm','GVEG','PotEvap','RootMoist','SoilM_100_200cm']:
    assert col in list(df.columns)

  scaled_data = StandardScaler().fit_transform(df[['Rainf', 'Evap','AvgSurfT','Albedo','SoilT_40_100cm','GVEG','PotEvap','RootMoist','SoilM_100_200cm']])
  scaled_df = pd.DataFrame(scaled_data, columns=['Rainf', 'Evap','AvgSurfT','Albedo','SoilT_40_100cm','GVEG','PotEvap','RootMoist','SoilM_100_200cm'])
  return pd.concat([df[["lon", "lat"]], scaled_df], axis=1)

In [11]:
def MinMaxScaling_on_predefined_attributes(df: pd.DataFrame) -> pd.DataFrame:
  """
  Funkcja służąca do przeskalowania kolumn 'Rainf', 'Evap','AvgSurfT','Albedo','SoilT_40_100cm','GVEG','PotEvap','RootMoist','SoilM_100_200cm' z ramki danych df za pomocą funkcji MinMaxScaler.
  Parametry:
  - df (DataFrame): Pandas DataFrame zawierająca co najmniej kolumny 'lon', 'lat', 'Rainf', 'Evap','AvgSurfT','Albedo','SoilT_40_100cm','GVEG','PotEvap','RootMoist','SoilM_100_200cm'.
  """
  for col in ['lon', 'lat', 'Rainf', 'Evap','AvgSurfT','Albedo','SoilT_40_100cm','GVEG','PotEvap','RootMoist','SoilM_100_200cm']:
    assert col in list(df.columns)

  scaled_data = MinMaxScaler().fit_transform(df[['Rainf', 'Evap','AvgSurfT','Albedo','SoilT_40_100cm','GVEG','PotEvap','RootMoist','SoilM_100_200cm']])
  scaled_df = pd.DataFrame(scaled_data, columns=['Rainf', 'Evap','AvgSurfT','Albedo','SoilT_40_100cm','GVEG','PotEvap','RootMoist','SoilM_100_200cm'])
  return pd.concat([df[["lon", "lat"]], scaled_df], axis=1)

In [12]:
def do_birch_and_return_lon_lat_cluster(scaled_df: pd.DataFrame, n_clusters: int=2, threshold: float=0.5, branching_factor: int=50) -> pd.DataFrame:
  """
  Funkcja wykonuje grupowanie algorytmem Birtch dla n_clusters klastrów oraz ustalonego parametru thershold. Funkcja zwraca ramkę danych zawierającą 'lon', 'lat' i wynik grupowania.
  Parametry:
  - scaled_df (DataFrame): Pandas DataFrame zawierająca 'lon', 'lat', 'Rainf', 'Evap','AvgSurfT','Albedo','SoilT_40_100cm','GVEG','PotEvap','RootMoist','SoilM_100_200cm',
  - n_clusters (int): maksymalna liczba klastrów,
  - threshold (float): promień klastra,
  - branching_factor (int): maksymalna liczba podklastrów CF w każdym węźle.
  """
  for col in ['lon', 'lat', 'Rainf', 'Evap','AvgSurfT','Albedo','SoilT_40_100cm','GVEG','PotEvap','RootMoist','SoilM_100_200cm']:
    assert col in list(scaled_df.columns)

  birch_model = Birch(threshold=threshold, branching_factor=branching_factor, n_clusters=n_clusters)
  scaled_df["Cluster"] = birch_model.fit_predict(scaled_df[['Rainf', 'Evap','AvgSurfT','Albedo','SoilT_40_100cm','GVEG','PotEvap','RootMoist','SoilM_100_200cm']])
  return scaled_df[["lon", "lat", "Cluster"]]

In [13]:
def show_metrics(y_pred: pd.DataFrame, y_true: pd.DataFrame) -> None:
  accuracy = accuracy_score(y_true, y_pred)
  print(f"Dokładność (accuracy): {accuracy*100:.2f}%")
  precision = precision_score(y_true, y_pred)
  print(f"Precyzja (precision): {precision*100:.2f}%")
  recall = recall_score(y_true, y_pred)
  print(f"Czułość (recall): {recall*100:.2f}%")
  f1 = f1_score(y_true, y_pred)
  print(f"F1-score: {f1*100:.2f}%")

In [14]:
'''
Funkcja jako argumenty bierze listę wartości określających granice przedziałów liczbowych, które
będą określać jak dla rozważanego parametru mają zmieniać się kolory punktów, których lista stanowi
drugi argument funkcji.
'''
def get_colormap(values: list, colors_palette: list, name = 'custom'):
    values = np.sort(np.array(values))
    values = np.interp(values, (values.min(), values.max()), (0, 1))
    cmap = mpl.colors.LinearSegmentedColormap.from_list(name, list(zip(values, colors_palette)))
    return cmap

In [15]:
def plot_map(df: pd.DataFrame, parameter_name: str, colormap: mpl.colors.LinearSegmentedColormap, title: str,
             point_size: int = 8, width: int = 800, height: int = 500, alpha: float = 1,
             bgcolor: str = 'white', colorbar_verbose: bool = False):

    gdf = gv.Points(df, ['lon', 'lat'], [parameter_name]) # obiekt zawierający punkty
    tiles = gts.OSM # wybór mapy tła, w tym wypadku OpenStreetMap

    # łączenie mapy tła z punktami i ustawienie wybranych parametrów wizualizacji
    map_with_points = tiles * gdf.opts(
        title=title,
        color=parameter_name,
        cmap=colormap,
        size=point_size,
        width=width,
        height=height,
        colorbar=colorbar_verbose,
        toolbar='above',
        tools=['hover', 'wheel_zoom', 'reset'],
        alpha=alpha, # przezroczystość
        bgcolor=bgcolor
    )

    return hv.render(map_with_points)

## Przygotowanie danych

Ponieważ poprzednie analizy pokazały, że grupowanie na podstawie miesięcy zimowych nie przynosi pozytywnych rezultatów, ograniczymy się do analizy miesięcy wiosennych, letnich i jesiennych.

In [16]:
# Ten sposób jest bardzo czasochłonny, więc zapisałem ramki danych jako csv
#nasa_202305 = load_single_month(spark, year=2023, month=5).toPandas()
#nasa_202306 = load_single_month(spark, year=2023, month=6).toPandas()
#nasa_202307 = load_single_month(spark, year=2023, month=7).toPandas()
#nasa_202308 = load_single_month(spark, year=2023, month=8).toPandas()
#nasa_202309 = load_single_month(spark, year=2023, month=9).toPandas()

In [17]:
#nasa_202305.to_csv("/content/drive/MyDrive/BigMess/NASA/NASA_birch_csv/nasa_202305", sep=';')
#nasa_202306.to_csv("/content/drive/MyDrive/BigMess/NASA/NASA_birch_csv/nasa_202306", sep=';')
#nasa_202307.to_csv("/content/drive/MyDrive/BigMess/NASA/NASA_birch_csv/nasa_202307", sep=';')
#nasa_202308.to_csv("/content/drive/MyDrive/BigMess/NASA/NASA_birch_csv/nasa_202308", sep=';')
#nasa_202309.to_csv("/content/drive/MyDrive/BigMess/NASA/NASA_birch_csv/nasa_202309", sep=';')

In [18]:
nasa_202305 = pd.read_csv('/content/drive/MyDrive/BigMess/NASA/NASA_birch_csv/nasa_202305',sep=';')
nasa_202306 = pd.read_csv('/content/drive/MyDrive/BigMess/NASA/NASA_birch_csv/nasa_202306',sep=';')
nasa_202307 = pd.read_csv('/content/drive/MyDrive/BigMess/NASA/NASA_birch_csv/nasa_202307',sep=';')
nasa_202308 = pd.read_csv('/content/drive/MyDrive/BigMess/NASA/NASA_birch_csv/nasa_202308',sep=';')
nasa_202309 = pd.read_csv('/content/drive/MyDrive/BigMess/NASA/NASA_birch_csv/nasa_202309',sep=';')

In [19]:
list_of_df = [nasa_202305, nasa_202306, nasa_202307, nasa_202308, nasa_202309]

In [20]:
NASA_an = pd.read_csv('/content/drive/MyDrive/BigMess/NASA/NASA_an.csv',sep=';')
NASA_an["pustynia_i_step"] = NASA_an["pustynia"] + NASA_an["step"]

Do grupowania będziemy używać następujących wskaźników:

*   **Rainf** - wskaźnik opadów deszczu,
*   **Evap** - wskaźnik całkowitej ewapotranspiracji,
*   **AvgSurfT** - wskaźnik średniej temperatury powierzchni ziemi,
*   **Albedo** - wskaźnik albedo,
*   **SoilT_40_100cm** - wskaźnik temperatury gleby w warstwie o głębokości od 40 do 100 cm,
*   **GVEG** - wskaźnik roślinności,
*   **PotEvap** - wskaźnik potencjalnej ewapotranspiracji,
*   **RootMoist** - wilgotność gleby w strefie korzeniowej (parowanie, które miałoby miejsce, gdyby dostępne było wystarczające źródło wody),
*   **SoilM_100_200cm** - wilgotność gleby w warstwie o głębokości od 100 do 200 cm.

# Grupowanie algorytmem Birch

**Birch** to oszczędzający pamięć algorytm do nauki online, stanowiący alternatywę dla MiniBatchKMeans. Konstruuje drzewiastą strukturę danych, w której centroidy klastrów są odczytywane z liścia.

## 1. Dane przeskalowane funkcją MinMaxScaler

Skalowanie danych

In [21]:
list_of_scaled_df = [MinMaxScaling_on_predefined_attributes(dataframe) for dataframe in list_of_df]

Grupowanie danych

In [22]:
list_of_results = [do_birch_and_return_lon_lat_cluster(scaled_df) for scaled_df in list_of_scaled_df]

Przeetykietowanie danych

In [23]:
list_of_results[4]["Cluster"] = list_of_results[4]["Cluster"].apply(lambda x: 1 if x == 0 else 0)

<ipython-input-23-0b530089f5cf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  list_of_results[4]["Cluster"] = list_of_results[4]["Cluster"].apply(lambda x: 1 if x == 0 else 0)


Wybranie tylko punktów zaanotowanych

In [24]:
list_of_merged_df = [
    df.merge(NASA_an[['lon', 'lat', 'pustynia', 'pustynia_i_step']], on=['lon','lat'], how='inner')
    for df
    in list_of_results
]

Mapa Stanów Zjednoczonych

In [25]:
display(IFrame("https://www.google.com/maps/embed?pb=!1m14!1m12!1m3!1d13982681.959428234!2d-98.66341902257437!3d38.39997874427714!2m3!1f0!2f0!3f0!3m2!1i1024!2i768!4f13.1!5e1!3m2!1spl!2spl!4v1703000232420!5m2!1spl!2spl", '800px', '500px'))

Mapa danych zaanotowanych, gdy step traktujemy jako teren niepustynny

In [26]:
colormap_cluster = get_colormap([0, max(list_of_merged_df[0]["pustynia"].values)], ['darkgreen', 'orange'])
output_notebook()
show(plot_map(df=list_of_merged_df[0], parameter_name='pustynia', colormap=colormap_cluster, title=f"Dane zaanotowane - step to teren niepustynny", alpha=0.5))

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)


Mapa danych zaanotowanych, gdy step traktujemy jako teren pustynny

In [27]:
colormap_cluster = get_colormap([0, max(list_of_merged_df[0]["pustynia_i_step"].values)], ['darkgreen', 'orange'])
output_notebook()
show(plot_map(df=list_of_merged_df[0], parameter_name='pustynia_i_step', colormap=colormap_cluster, title=f"Dane zaanotowane - step to teren pustynny", alpha=0.5))

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)


Prezentacja otrzymanych klastrów na mapach - tylko punkty zaanotowane

In [28]:
for df, col_name in zip(list_of_merged_df, list_of_df):
  colormap_cluster = get_colormap([0, max(df["Cluster"].values)], ['darkgreen', 'orange'])
  output_notebook()
  show(plot_map(df=df, parameter_name='Cluster', colormap=colormap_cluster, title=f"Wyniki grupowania dla danych z {'0'+str(col_name.loc[0,'Month'])}.{col_name.loc[0,'Year']}", alpha=0.5))

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)


/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)


/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)


/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)


/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)


### Wyświetlenie metryk, gdy step traktujemy jako teren niepustynny

Dane z 05.2023

In [29]:
show_metrics(list_of_merged_df[0]['Cluster'], list_of_merged_df[0]['pustynia'])

Dokładność (accuracy): 37.52%
Precyzja (precision): 0.34%
Czułość (recall): 1.26%
F1-score: 0.53%


Dane z 06.2023

In [30]:
show_metrics(list_of_merged_df[1]['Cluster'], list_of_merged_df[1]['pustynia'])

Dokładność (accuracy): 87.94%
Precyzja (precision): 53.97%
Czułość (recall): 64.15%
F1-score: 58.62%


Dane z 07.2023

In [31]:
show_metrics(list_of_merged_df[2]['Cluster'], list_of_merged_df[2]['pustynia'])

Dokładność (accuracy): 79.23%
Precyzja (precision): 39.01%
Czułość (recall): 99.37%
F1-score: 56.03%


Dane z 08.2023

In [32]:
show_metrics(list_of_merged_df[3]['Cluster'], list_of_merged_df[3]['pustynia'])

Dokładność (accuracy): 73.53%
Precyzja (precision): 33.40%
Czułość (recall): 99.37%
F1-score: 50.00%


Dane z 09.2023

In [33]:
show_metrics(list_of_merged_df[4]['Cluster'], list_of_merged_df[4]['pustynia'])

Dokładność (accuracy): 73.45%
Precyzja (precision): 33.19%
Czułość (recall): 98.11%
F1-score: 49.60%


### Wyświetlenie metryk, gdy step traktujemy jako teren pustynny

Dane z 05.2023

In [34]:
show_metrics(list_of_merged_df[0]['Cluster'], list_of_merged_df[0]['pustynia_i_step'])

Dokładność (accuracy): 26.05%
Precyzja (precision): 3.72%
Czułość (recall): 6.55%
F1-score: 4.75%


Dane z 06.2023

In [35]:
show_metrics(list_of_merged_df[1]['Cluster'], list_of_merged_df[1]['pustynia_i_step'])

Dokładność (accuracy): 81.32%
Precyzja (precision): 79.89%
Czułość (recall): 44.94%
F1-score: 57.52%


Dane z 07.2023

In [36]:
show_metrics(list_of_merged_df[2]['Cluster'], list_of_merged_df[2]['pustynia_i_step'])

Dokładność (accuracy): 87.69%
Precyzja (precision): 73.33%
Czułość (recall): 88.39%
F1-score: 80.16%


Dane z 08.2023

In [37]:
show_metrics(list_of_merged_df[3]['Cluster'], list_of_merged_df[3]['pustynia_i_step'])

Dokładność (accuracy): 83.50%
Precyzja (precision): 64.69%
Czułość (recall): 91.07%
F1-score: 75.65%


Dane z 09.2023

In [38]:
show_metrics(list_of_merged_df[4]['Cluster'], list_of_merged_df[4]['pustynia_i_step'])

Dokładność (accuracy): 82.58%
Precyzja (precision): 63.62%
Czułość (recall): 88.99%
F1-score: 74.19%


## 2. Dane przeskalowane funkcją StandardScaler

Skalowanie danych

In [39]:
list_of_scaled_df = [StandardScaling_on_predefined_attributes(dataframe) for dataframe in list_of_df]

Grupowanie danych

In [40]:
list_of_results = [do_birch_and_return_lon_lat_cluster(scaled_df) for scaled_df in list_of_scaled_df]

Przeetykietowanie danych

In [41]:
list_of_results[0]["Cluster"] = list_of_results[0]["Cluster"].apply(lambda x: 1 if x == 0 else 0)
list_of_results[1]["Cluster"] = list_of_results[1]["Cluster"].apply(lambda x: 1 if x == 0 else 0)
list_of_results[3]["Cluster"] = list_of_results[3]["Cluster"].apply(lambda x: 1 if x == 0 else 0)
list_of_results[4]["Cluster"] = list_of_results[4]["Cluster"].apply(lambda x: 1 if x == 0 else 0)

<ipython-input-41-904039db65ae>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  list_of_results[0]["Cluster"] = list_of_results[0]["Cluster"].apply(lambda x: 1 if x == 0 else 0)
<ipython-input-41-904039db65ae>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  list_of_results[1]["Cluster"] = list_of_results[1]["Cluster"].apply(lambda x: 1 if x == 0 else 0)
<ipython-input-41-904039db65ae>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

Wybranie tylko punktów zaanotowanych

In [42]:
list_of_merged_df = [
    df.merge(NASA_an[['lon', 'lat', 'pustynia', 'pustynia_i_step']], on=['lon','lat'], how='inner')
    for df
    in list_of_results
]

Mapa danych zaanotowanych, gdy step traktujemy jako teren niepustynny

In [43]:
colormap_cluster = get_colormap([0, max(list_of_merged_df[0]["pustynia"].values)], ['darkgreen', 'orange'])
output_notebook()
show(plot_map(df=list_of_merged_df[0], parameter_name='pustynia', colormap=colormap_cluster, title=f"Dane zaanotowane - step to teren niepustynny", alpha=0.5))

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)


Mapa danych zaanotowanych, gdy step traktujemy jako teren pustynny

In [44]:
colormap_cluster = get_colormap([0, max(list_of_merged_df[0]["pustynia_i_step"].values)], ['darkgreen', 'orange'])
output_notebook()
show(plot_map(df=list_of_merged_df[0], parameter_name='pustynia_i_step', colormap=colormap_cluster, title=f"Dane zaanotowane - step to teren pustynny", alpha=0.5))

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)


Prezentacja otrzymanych klastrów na mapach - tylko punkty zaanotowane

In [45]:
for df, col_name in zip(list_of_merged_df, list_of_df):
  colormap_cluster = get_colormap([0, max(df["Cluster"].values)], ['darkgreen', 'orange'])
  output_notebook()
  show(plot_map(df=df, parameter_name='Cluster', colormap=colormap_cluster, title=f"Wyniki grupowania dla danych z {'0'+str(col_name.loc[0,'Month'])}.{col_name.loc[0,'Year']}", alpha=0.5))

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)


/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)


/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)


/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)


/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)


### Wyświetlenie metryk, gdy step traktujemy jako teren niepustynny

Dane z 05.2023

In [46]:
show_metrics(list_of_merged_df[0]['Cluster'], list_of_merged_df[0]['pustynia'])

Dokładność (accuracy): 71.02%
Precyzja (precision): 26.33%
Czułość (recall): 65.41%
F1-score: 37.55%


Dane z 06.2023

In [47]:
show_metrics(list_of_merged_df[1]['Cluster'], list_of_merged_df[1]['pustynia'])

Dokładność (accuracy): 51.09%
Precyzja (precision): 21.09%
Czułość (recall): 97.48%
F1-score: 34.68%


Dane z 07.2023

In [48]:
show_metrics(list_of_merged_df[2]['Cluster'], list_of_merged_df[2]['pustynia'])

Dokładność (accuracy): 82.50%
Precyzja (precision): 42.86%
Czułość (recall): 94.34%
F1-score: 58.94%


Dane z 08.2023

In [49]:
show_metrics(list_of_merged_df[3]['Cluster'], list_of_merged_df[3]['pustynia'])

Dokładność (accuracy): 48.07%
Precyzja (precision): 20.41%
Czułość (recall): 100.00%
F1-score: 33.90%


Dane z 09.2023

In [50]:
show_metrics(list_of_merged_df[4]['Cluster'], list_of_merged_df[4]['pustynia'])

Dokładność (accuracy): 58.71%
Precyzja (precision): 13.54%
Czułość (recall): 38.99%
F1-score: 20.10%


### Wyświetlenie metryk, gdy step traktujemy jako teren pustynny

Dane z 05.2023

In [51]:
show_metrics(list_of_merged_df[0]['Cluster'], list_of_merged_df[0]['pustynia_i_step'])

Dokładność (accuracy): 66.25%
Precyzja (precision): 41.52%
Czułość (recall): 48.81%
F1-score: 44.87%


Dane z 06.2023

In [52]:
show_metrics(list_of_merged_df[1]['Cluster'], list_of_merged_df[1]['pustynia_i_step'])

Dokładność (accuracy): 61.39%
Precyzja (precision): 41.50%
Czułość (recall): 90.77%
F1-score: 56.96%


Dane z 07.2023

In [53]:
show_metrics(list_of_merged_df[2]['Cluster'], list_of_merged_df[2]['pustynia_i_step'])

Dokładność (accuracy): 85.59%
Precyzja (precision): 73.43%
Czułość (recall): 76.49%
F1-score: 74.93%


Dane z 08.2023

In [54]:
show_metrics(list_of_merged_df[3]['Cluster'], list_of_merged_df[3]['pustynia_i_step'])

Dokładność (accuracy): 60.72%
Precyzja (precision): 41.46%
Czułość (recall): 96.13%
F1-score: 57.94%


Dane z 09.2023

In [55]:
show_metrics(list_of_merged_df[4]['Cluster'], list_of_merged_df[4]['pustynia_i_step'])

Dokładność (accuracy): 52.76%
Precyzja (precision): 25.11%
Czułość (recall): 34.23%
F1-score: 28.97%


# Tunning hiperparametrów modelu trenowanego na danych skalowanych funkcją StandardScaler z lipca 2023

## Threshold: 0.4, branching factor: 45

Gdy step to teren niepustynny

In [89]:
result = do_birch_and_return_lon_lat_cluster(list_of_scaled_df[2], threshold=0.4, branching_factor=45)
merged_result = result.merge(NASA_an[['lon', 'lat', 'pustynia', 'pustynia_i_step']], on=['lon','lat'], how='inner')
show_metrics(merged_result['Cluster'], merged_result['pustynia'])

Dokładność (accuracy): 65.33%
Precyzja (precision): 27.75%
Czułość (recall): 100.00%
F1-score: 43.44%


In [90]:
colormap_cluster = get_colormap([0, max(merged_result["Cluster"].values)], ['darkgreen', 'orange'])
output_notebook()
show(plot_map(df=merged_result, parameter_name='Cluster', colormap=colormap_cluster, title="Threshold: 0.4, branching factor: 45", alpha=0.5))

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)


Gdy step to teren pustynny

In [91]:
show_metrics(merged_result['Cluster'], merged_result['pustynia_i_step'])

Dokładność (accuracy): 75.96%
Precyzja (precision): 54.28%
Czułość (recall): 92.56%
F1-score: 68.43%


## Threshold: 0.4, branching factor: 50

Gdy step to teren niepustynny

In [92]:
result = do_birch_and_return_lon_lat_cluster(list_of_scaled_df[2], threshold=0.4, branching_factor=50)
merged_result = result.merge(NASA_an[['lon', 'lat', 'pustynia', 'pustynia_i_step']], on=['lon','lat'], how='inner')
show_metrics(merged_result['Cluster'], merged_result['pustynia'])

Dokładność (accuracy): 86.77%
Precyzja (precision): 50.18%
Czułość (recall): 86.16%
F1-score: 63.43%


In [93]:
colormap_cluster = get_colormap([0, max(merged_result["Cluster"].values)], ['darkgreen', 'orange'])
output_notebook()
show(plot_map(df=merged_result, parameter_name='Cluster', colormap=colormap_cluster, title="Threshold: 0.4, branching factor: 50", alpha=0.5))

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)


Gdy step to teren pustynny

In [94]:
show_metrics(merged_result['Cluster'], merged_result['pustynia_i_step'])

Dokładność (accuracy): 84.67%
Precyzja (precision): 78.02%
Czułość (recall): 63.39%
F1-score: 69.95%


## Threshold: 0.4, branching factor: 55

Gdy step to teren niepustynny

In [95]:
result = do_birch_and_return_lon_lat_cluster(list_of_scaled_df[2], threshold=0.4, branching_factor=55)
merged_result = result.merge(NASA_an[['lon', 'lat', 'pustynia', 'pustynia_i_step']], on=['lon','lat'], how='inner')
show_metrics(merged_result['Cluster'], merged_result['pustynia'])

Dokładność (accuracy): 77.30%
Precyzja (precision): 36.47%
Czułość (recall): 94.97%
F1-score: 52.71%


In [96]:
colormap_cluster = get_colormap([0, max(merged_result["Cluster"].values)], ['darkgreen', 'orange'])
output_notebook()
show(plot_map(df=merged_result, parameter_name='Cluster', colormap=colormap_cluster, title="Threshold: 0.4, branching factor: 55", alpha=0.5))

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)


Gdy step to teren pustynny

In [97]:
show_metrics(merged_result['Cluster'], merged_result['pustynia_i_step'])

Dokładność (accuracy): 83.25%
Precyzja (precision): 66.43%
Czułość (recall): 81.85%
F1-score: 73.33%


## Threshold: 0.5, branching factor: 45

Gdy step to teren niepustynny

In [98]:
result = do_birch_and_return_lon_lat_cluster(list_of_scaled_df[2], threshold=0.5, branching_factor=45)
merged_result = result.merge(NASA_an[['lon', 'lat', 'pustynia', 'pustynia_i_step']], on=['lon','lat'], how='inner')
show_metrics(merged_result['Cluster'], merged_result['pustynia'])

Dokładność (accuracy): 63.90%
Precyzja (precision): 26.63%
Czułość (recall): 97.48%
F1-score: 41.84%


In [99]:
colormap_cluster = get_colormap([0, max(merged_result["Cluster"].values)], ['darkgreen', 'orange'])
output_notebook()
show(plot_map(df=merged_result, parameter_name='Cluster', colormap=colormap_cluster, title="Threshold: 0.5, branching factor: 45", alpha=0.5))

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)


Gdy step to teren pustynny

In [100]:
show_metrics(merged_result['Cluster'], merged_result['pustynia_i_step'])

Dokładność (accuracy): 73.37%
Precyzja (precision): 51.55%
Czułość (recall): 89.29%
F1-score: 65.36%


## Threshold: 0.5, branching factor: 55

Gdy step to teren niepustynny

In [101]:
result = do_birch_and_return_lon_lat_cluster(list_of_scaled_df[2], threshold=0.5, branching_factor=55)
merged_result = result.merge(NASA_an[['lon', 'lat', 'pustynia', 'pustynia_i_step']], on=['lon','lat'], how='inner')
show_metrics(merged_result['Cluster'], merged_result['pustynia'])

Dokładność (accuracy): 86.77%
Precyzja (precision): 50.31%
Czułość (recall): 50.31%
F1-score: 50.31%


In [102]:
colormap_cluster = get_colormap([0, max(merged_result["Cluster"].values)], ['darkgreen', 'orange'])
output_notebook()
show(plot_map(df=merged_result, parameter_name='Cluster', colormap=colormap_cluster, title="Threshold: 0.5, branching factor: 55", alpha=0.5))

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)


Gdy step to teren pustynny

In [103]:
show_metrics(merged_result['Cluster'], merged_result['pustynia_i_step'])

Dokładność (accuracy): 79.15%
Precyzja (precision): 77.36%
Czułość (recall): 36.61%
F1-score: 49.70%


## Threshold: 0.6, branching factor: 45

Gdy step to teren niepustynny

In [104]:
result = do_birch_and_return_lon_lat_cluster(list_of_scaled_df[2], threshold=0.6, branching_factor=45)
merged_result = result.merge(NASA_an[['lon', 'lat', 'pustynia', 'pustynia_i_step']], on=['lon','lat'], how='inner')
show_metrics(merged_result['Cluster'], merged_result['pustynia'])

Dokładność (accuracy): 84.42%
Precyzja (precision): 45.33%
Czułość (recall): 82.39%
F1-score: 58.48%


In [105]:
colormap_cluster = get_colormap([0, max(merged_result["Cluster"].values)], ['darkgreen', 'orange'])
output_notebook()
show(plot_map(df=merged_result, parameter_name='Cluster', colormap=colormap_cluster, title="Threshold: 0.6, branching factor: 45", alpha=0.5))

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)


Gdy step to teren pustynny

In [106]:
show_metrics(merged_result['Cluster'], merged_result['pustynia_i_step'])

Dokładność (accuracy): 84.51%
Precyzja (precision): 76.12%
Czułość (recall): 65.48%
F1-score: 70.40%


## Threshold: 0.6, branching factor: 50

Gdy step to teren niepustynny

In [107]:
result = do_birch_and_return_lon_lat_cluster(list_of_scaled_df[2], threshold=0.6, branching_factor=50)
merged_result = result.merge(NASA_an[['lon', 'lat', 'pustynia', 'pustynia_i_step']], on=['lon','lat'], how='inner')
show_metrics(merged_result['Cluster'], merged_result['pustynia'])

Dokładność (accuracy): 86.18%
Precyzja (precision): 48.74%
Czułość (recall): 72.96%
F1-score: 58.44%


In [108]:
colormap_cluster = get_colormap([0, max(merged_result["Cluster"].values)], ['darkgreen', 'orange'])
output_notebook()
show(plot_map(df=merged_result, parameter_name='Cluster', colormap=colormap_cluster, title="Threshold: 0.6, branching factor: 50", alpha=0.5))

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)


Gdy step to teren pustynny

In [109]:
show_metrics(merged_result['Cluster'], merged_result['pustynia_i_step'])

Dokładność (accuracy): 82.41%
Precyzja (precision): 76.47%
Czułość (recall): 54.17%
F1-score: 63.41%


## Threshold: 0.6, branching factor: 55

Gdy step to teren niepustynny

In [110]:
result = do_birch_and_return_lon_lat_cluster(list_of_scaled_df[2], threshold=0.6, branching_factor=55)
merged_result = result.merge(NASA_an[['lon', 'lat', 'pustynia', 'pustynia_i_step']], on=['lon','lat'], how='inner')
show_metrics(merged_result['Cluster'], merged_result['pustynia'])

Dokładność (accuracy): 71.94%
Precyzja (precision): 31.97%
Czułość (recall): 98.11%
F1-score: 48.22%


In [111]:
colormap_cluster = get_colormap([0, max(merged_result["Cluster"].values)], ['darkgreen', 'orange'])
output_notebook()
show(plot_map(df=merged_result, parameter_name='Cluster', colormap=colormap_cluster, title="Threshold: 0.6, branching factor: 55", alpha=0.5))

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)


Gdy step to teren pustynny

In [112]:
show_metrics(merged_result['Cluster'], merged_result['pustynia_i_step'])

Dokładność (accuracy): 80.40%
Precyzja (precision): 60.45%
Czułość (recall): 87.80%
F1-score: 71.60%


In [ ]:
!jupyter nbconvert --to html "/content/Birch.ipynb"